In [0]:
"""
For each region find house which has won maximum no of battles. display region, house and no of wins
output columns: house, region, no of wins
"""

from pyspark.sql.functions import *
from pyspark.sql.window import Window

king = spark.createDataFrame([
    (1, 'Robb Stark', 'House Stark'),
    (2, 'Joffrey Baratheon', 'House Lannister'),
    (3, 'Stannis Baratheon', 'House Baratheon'),
    (4, 'Balon Greyjoy', 'House Greyjoy'),
    (5, 'Mace Tyrell', 'House Tyrell'),
    (6, 'Doran Martell', 'House Martell')
], ["k_no", "king", "house"])

battle = spark.createDataFrame([
    (1, 'Battle of Oxcross', 1, 2, 1, 'The North'),
    (2, 'Battle of Blackwater', 3, 4, 0, 'The North'),
    (4, 'Battle of the Green Fork', 2, 6, 0, 'The Reach'),
    (5, 'Battle of the Ruby Ford', 1, 3, 1, 'The Riverlands'),
    (6, 'Battle of the Golden Tooth', 2, 1, 0, 'The North'),
    (7, 'Battle of Riverrun', 3, 4, 1, 'The Riverlands'),
    (8, 'Battle of Riverrun', 1, 3, 0, 'The Riverlands')
], ["battle_number", "name", "attacker_king", "defender_king", "attacker_outcome", "region"])

king.show(truncate=False)
battle.show(truncate=False)
king.printSchema()
battle.printSchema()

+----+-----------------+---------------+
|k_no|king             |house          |
+----+-----------------+---------------+
|1   |Robb Stark       |House Stark    |
|2   |Joffrey Baratheon|House Lannister|
|3   |Stannis Baratheon|House Baratheon|
|4   |Balon Greyjoy    |House Greyjoy  |
|5   |Mace Tyrell      |House Tyrell   |
|6   |Doran Martell    |House Martell  |
+----+-----------------+---------------+

+-------------+--------------------------+-------------+-------------+----------------+--------------+
|battle_number|name                      |attacker_king|defender_king|attacker_outcome|region        |
+-------------+--------------------------+-------------+-------------+----------------+--------------+
|1            |Battle of Oxcross         |1            |2            |1               |The North     |
|2            |Battle of Blackwater      |3            |4            |0               |The North     |
|4            |Battle of the Green Fork  |2            |6            |0   

In [0]:
"""
The Reach House Stark is not there in the output not sure why
"""

battle1 = battle.withColumn("battle_king_no", when(col("attacker_king") == 1, col("attacker_king")).otherwise(col("defender_king")))
    
battle1.join(king, battle1["battle_king_no"] == king["k_no"], "inner").orderBy("battle_number")\
    .groupBy("region", "house").agg(count("region").alias("no_of_wins")) \
    .withColumn("rn", rank().over(Window.partitionBy("region").orderBy(desc(col("no_of_wins"))))) \
    .filter(col("rn") == 1)\
    .drop("rn") \
    .show() 

+--------------+-------------+----------+
|        region|        house|no_of_wins|
+--------------+-------------+----------+
|     The North|  House Stark|         2|
|     The Reach|House Martell|         1|
|The Riverlands|  House Stark|         2|
+--------------+-------------+----------+

